# EIP-8037: Quantization Analysis

Test different values for `CPSB_SIGNIFICANT_BITS` and `CPSB_OFFSET` and reproduce the boundary distance table.

In [11]:
import math

In [12]:

def compute_cpsb(gas_limit, sig_bits, offset, target_state_growth_per_year):
    """Compute cost_per_state_byte for a given gas limit and quantization parameters."""
    raw = math.ceil((gas_limit * 2_628_000) / (2 * target_state_growth_per_year))
    shifted = raw + offset
    shift = max(0, shifted.bit_length() - sig_bits)
    cost = max(((shifted >> shift) << shift) - offset, 1)
    return cost


def find_boundary_distance(target_gl, sig_bits, offset, target_state_growth_per_year):
    """Find the distance from target_gl to the nearest quantization boundary.

    Returns (distance_pct, blocks_to_boundary, cpsb_value).
    """
    p = 2_628_000
    q = 2 * target_state_growth_per_year
    # Calculate shifted, shift and cpsb
    raw = math.ceil(target_gl * p / q)
    shifted = raw + offset
    shift = max(0, shifted.bit_length() - sig_bits)
    cpsb_at_target = max(((shifted >> shift) << shift) - offset, 1)
    # Bucket boundaries in shifted-space
    lower_shifted = (shifted >> shift) << shift
    upper_shifted = lower_shifted + (1 << shift)
    # Smallest gl where ceil(gl * p/q) >= n is: (n-1) * q // p + 1
    gl_upper = (upper_shifted - offset - 1) * q // p + 1
    gl_lower = (lower_shifted - offset - 1) * q // p + 1
    # Compute metrics
    dist_up = gl_upper - target_gl
    dist_down = target_gl - gl_lower
    min_dist = min(dist_up, dist_down)
    max_change_per_block = target_gl / 1024
    return (
        min_dist / target_gl * 100,
        min_dist / max_change_per_block,
        cpsb_at_target,
    )


## Brute-force search for best offset

In [13]:
TARGET_STATE_GROWTH_PER_YEAR = 100 * 1024**3

In [14]:
# Search range for offset
SEARCH_BITS = 4
SEARCH_MAX_OFFSET = 10000
SEARCH_GAS_LIMITS = [gl_m * 1_000_000 for gl_m in range(100, 1050, 50)]

best = []
for offset in range(SEARCH_MAX_OFFSET + 1):
    min_frac = float("inf")
    worst = 0
    for gl in SEARCH_GAS_LIMITS:
        frac, _, _ = find_boundary_distance(gl, SEARCH_BITS, offset, TARGET_STATE_GROWTH_PER_YEAR)
        if frac < min_frac:
            min_frac = frac
            worst = gl
    best.append((min_frac, offset, worst))

best.sort(reverse=True)

print(f"Top 10 offsets for CPSB_SIGNIFICANT_BITS={SEARCH_BITS}:")
print("")
print(f"{'Rank':>4} {'Offset':>8} {'Min dist':>10} {'Worst GL':>10} {'Blocks':>8}")
print("-" * 44)
for i, (frac, offset, worst_gl) in enumerate(best[:10]):
    _, blk, _ = find_boundary_distance(worst_gl, SEARCH_BITS, offset, TARGET_STATE_GROWTH_PER_YEAR)
    print(f"  #{i+1:<2} {offset:>8} {frac:>9.3f}% {worst_gl//1_000_000:>8}M {blk:>7.1f}")

Top 10 offsets for CPSB_SIGNIFICANT_BITS=4:

Rank   Offset   Min dist   Worst GL   Blocks
--------------------------------------------
  #1      9746     1.386%      700M    14.2
  #2      7698     1.386%      700M    14.2
  #3      9745     1.377%      550M    14.1
  #4      7697     1.377%      550M    14.1
  #5      9747     1.374%      700M    14.1
  #6      7699     1.374%      700M    14.1
  #7      9748     1.362%      700M    13.9
  #8      7700     1.362%      700M    13.9
  #9      9744     1.362%      550M    13.9
  #10     7696     1.362%      550M    13.9


In [15]:
# Search range for offset
SEARCH_BITS = 5
SEARCH_MAX_OFFSET = 10000
SEARCH_GAS_LIMITS = [gl_m * 1_000_000 for gl_m in range(100, 1050, 50)]

best = []
for offset in range(SEARCH_MAX_OFFSET + 1):
    min_frac = float("inf")
    worst = 0
    for gl in SEARCH_GAS_LIMITS:
        frac, _, _ = find_boundary_distance(gl, SEARCH_BITS, offset, TARGET_STATE_GROWTH_PER_YEAR)
        if frac < min_frac:
            min_frac = frac
            worst = gl
    best.append((min_frac, offset, worst))

best.sort(reverse=True)

print(f"Top 10 offsets for CPSB_SIGNIFICANT_BITS={SEARCH_BITS}:")
print("")
print(f"{'Rank':>4} {'Offset':>8} {'Min dist':>10} {'Worst GL':>10} {'Blocks':>8}")
print("-" * 44)
for i, (frac, offset, worst_gl) in enumerate(best[:10]):
    _, blk, _ = find_boundary_distance(worst_gl, SEARCH_BITS, offset, TARGET_STATE_GROWTH_PER_YEAR)
    print(f"  #{i+1:<2} {offset:>8} {frac:>9.3f}% {worst_gl//1_000_000:>8}M {blk:>7.1f}")

Top 10 offsets for CPSB_SIGNIFICANT_BITS=5:

Rank   Offset   Min dist   Worst GL   Blocks
--------------------------------------------
  #1      9578     0.888%      800M     9.1
  #2      9579     0.878%      800M     9.0
  #3      9580     0.868%      800M     8.9
  #4      9577     0.867%      350M     8.9
  #5      9581     0.857%      800M     8.8
  #6      9582     0.847%      800M     8.7
  #7      9576     0.844%      350M     8.6
  #8      9583     0.837%      800M     8.6
  #9      9584     0.827%      800M     8.5
  #10     9575     0.821%      350M     8.4


## Configure parameters

Change these values to test different quantization settings:

In [16]:
# === CHANGE THESE TO TEST DIFFERENT VALUES ===
CPSB_SIGNIFICANT_BITS = 5
CPSB_OFFSET = 9578

# Gas limits to evaluate (in millions)
GAS_LIMITS_M = list(range(100, 1050, 50))

## Boundary distance table

In [17]:
print(f"CPSB_SIGNIFICANT_BITS = {CPSB_SIGNIFICANT_BITS}")
print(f"CPSB_OFFSET = {CPSB_OFFSET}")
print()
print(f"{'Gas Limit':>12} {'cpsb':>8} {'Distance':>10} {'Blocks':>8}")
print("-" * 42)

worst_blocks = float("inf")
worst_gl = 0

for gl_m in GAS_LIMITS_M:
    gl = gl_m * 1_000_000
    dist_pct, blocks, cpsb = find_boundary_distance(gl, CPSB_SIGNIFICANT_BITS, CPSB_OFFSET, TARGET_STATE_GROWTH_PER_YEAR)
    flag = ""
    if blocks < worst_blocks:
        worst_blocks = blocks
        worst_gl = gl_m
    if blocks < 10:
        flag = " <-- warning"
    print(f"{gl_m:>9}M {cpsb:>8,} {dist_pct:>9.2f}% {blocks:>7.1f}{flag}")

print()
print(f"Worst case: {worst_gl}M with {worst_blocks:.1f} blocks to boundary")

CPSB_SIGNIFICANT_BITS = 5
CPSB_OFFSET = 9578

   Gas Limit     cpsb   Distance   Blocks
------------------------------------------
      100M    1,174      4.15%    42.5
      150M    1,686      8.21%    84.0
      200M    2,198     10.24%   104.8
      250M    2,710      5.28%    54.1
      300M    3,222      1.68%    17.2
      350M    4,246      0.89%     9.1 <-- warning
      400M    4,758      2.82%    28.9
      450M    5,270      4.32%    44.2
      500M    5,782      2.85%    29.2
      550M    6,294      1.10%    11.3
      600M    6,806      6.63%    67.8
      650M    7,830      1.58%    16.1
      700M    7,830      3.35%    34.3
      750M    8,854      3.54%    36.3
      800M    8,854      0.89%     9.1 <-- warning
      850M    9,878      4.80%    49.1
      900M   10,902      1.02%    10.5
      950M   10,902      2.57%    26.4
     1000M   11,926      2.55%    26.2

Worst case: 800M with 9.1 blocks to boundary


## Step size analysis

Shows how much `cost_per_state_byte` jumps when crossing a quantization boundary.

In [18]:
print(f"CPSB_SIGNIFICANT_BITS = {CPSB_SIGNIFICANT_BITS}")
print(f"CPSB_OFFSET = {CPSB_OFFSET}")
print()
print(f"{'Gas Limit':>12} {'cpsb':>8} {'Next cpsb':>10} {'Step':>8} {'Step %':>8}")
print("-" * 50)

for gl_m in GAS_LIMITS_M:
    gl = gl_m * 1_000_000
    cpsb = compute_cpsb(gl, CPSB_SIGNIFICANT_BITS, CPSB_OFFSET, TARGET_STATE_GROWTH_PER_YEAR)
    # Find next distinct cpsb value above
    gl_up = gl
    while compute_cpsb(gl_up, CPSB_SIGNIFICANT_BITS, CPSB_OFFSET, TARGET_STATE_GROWTH_PER_YEAR) == cpsb:
        gl_up += 1000
    next_cpsb = compute_cpsb(gl_up, CPSB_SIGNIFICANT_BITS, CPSB_OFFSET, TARGET_STATE_GROWTH_PER_YEAR)
    step = next_cpsb - cpsb
    step_pct = step / cpsb * 100
    print(f"{gl_m:>9}M {cpsb:>8,} {next_cpsb:>10,} {step:>8,} {step_pct:>7.1f}%")

CPSB_SIGNIFICANT_BITS = 5
CPSB_OFFSET = 9578

   Gas Limit     cpsb  Next cpsb     Step   Step %
--------------------------------------------------
      100M    1,174      1,686      512    43.6%
      150M    1,686      2,198      512    30.4%
      200M    2,198      2,710      512    23.3%
      250M    2,710      3,222      512    18.9%
      300M    3,222      3,734      512    15.9%
      350M    4,246      4,758      512    12.1%
      400M    4,758      5,270      512    10.8%
      450M    5,270      5,782      512     9.7%
      500M    5,782      6,294      512     8.9%
      550M    6,294      6,806      512     8.1%
      600M    6,806      7,830    1,024    15.0%
      650M    7,830      8,854    1,024    13.1%
      700M    7,830      8,854    1,024    13.1%
      750M    8,854      9,878    1,024    11.6%
      800M    8,854      9,878    1,024    11.6%
      850M    9,878     10,902    1,024    10.4%
      900M   10,902     11,926    1,024     9.4%
      950M   10,902

## Markdown tables (copy-paste into EIP)

In [19]:
print("| Gas Limit | `cost_per_state_byte` | Distance to boundary | Blocks to boundary |")
print("| :---------: | :---------------------: | :--------------------: | :------------------: |")
for gl_m in GAS_LIMITS_M:
    gl = gl_m * 1_000_000
    dist_pct, blocks, cpsb = find_boundary_distance(gl, CPSB_SIGNIFICANT_BITS, CPSB_OFFSET, TARGET_STATE_GROWTH_PER_YEAR)
    print(f"| {gl_m}M | {cpsb:,} | {dist_pct:.2f}% | {blocks:.1f} |")

| Gas Limit | `cost_per_state_byte` | Distance to boundary | Blocks to boundary |
| :---------: | :---------------------: | :--------------------: | :------------------: |
| 100M | 1,174 | 4.15% | 42.5 |
| 150M | 1,686 | 8.21% | 84.0 |
| 200M | 2,198 | 10.24% | 104.8 |
| 250M | 2,710 | 5.28% | 54.1 |
| 300M | 3,222 | 1.68% | 17.2 |
| 350M | 4,246 | 0.89% | 9.1 |
| 400M | 4,758 | 2.82% | 28.9 |
| 450M | 5,270 | 4.32% | 44.2 |
| 500M | 5,782 | 2.85% | 29.2 |
| 550M | 6,294 | 1.10% | 11.3 |
| 600M | 6,806 | 6.63% | 67.8 |
| 650M | 7,830 | 1.58% | 16.1 |
| 700M | 7,830 | 3.35% | 34.3 |
| 750M | 8,854 | 3.54% | 36.3 |
| 800M | 8,854 | 0.89% | 9.1 |
| 850M | 9,878 | 4.80% | 49.1 |
| 900M | 10,902 | 1.02% | 10.5 |
| 950M | 10,902 | 2.57% | 26.4 |
| 1000M | 11,926 | 2.55% | 26.2 |


## Estimated price impacts at various block gas limits

In [20]:
CURRENT_NEW_ACCOUNT = 25_000
CURRENT_NEW_SLOT = 20_000
CONTRACT_LEN = 24_576  # 24kB (EIP-170 limit)
CURRENT_DEPLOY = 32_000 + 200 * CONTRACT_LEN  # GAS_CREATE + GAS_CODE_DEPOSIT * L
ACCOUNT_BYTES = 112
SLOT_BYTES = 32
HASH_COST = 6 * math.ceil(CONTRACT_LEN / 32)
GAS_CREATE_REGULAR = 9_000
GAS_STORAGE_SET_REGULAR = 2_900  # GAS_STORAGE_UPDATE - GAS_COLD_SLOAD

IMPACT_GAS_LIMITS_M = [60, 100, 200, 300]

rows = []
rows.append(f"| Current cost | NA | {CURRENT_NEW_ACCOUNT:,} | NA | {CURRENT_NEW_SLOT:,} | NA | {CURRENT_DEPLOY:,} | NA |")

for gl_m in IMPACT_GAS_LIMITS_M:
    gl = gl_m * 1_000_000
    cpsb = compute_cpsb(gl, CPSB_SIGNIFICANT_BITS, CPSB_OFFSET, TARGET_STATE_GROWTH_PER_YEAR)
    new_account = ACCOUNT_BYTES * cpsb
    new_slot = SLOT_BYTES * cpsb + GAS_STORAGE_SET_REGULAR
    deploy = cpsb * (ACCOUNT_BYTES + CONTRACT_LEN) + GAS_CREATE_REGULAR + HASH_COST
    inc_account = round(new_account / CURRENT_NEW_ACCOUNT)
    inc_slot = round(new_slot / CURRENT_NEW_SLOT)
    inc_deploy = round(deploy / CURRENT_DEPLOY)
    rows.append(
        f"| Proposed cost at {gl_m}M block limit | {cpsb:,} "
        f"| {new_account:,} | {inc_account} "
        f"| {new_slot:,} | {inc_slot} "
        f"| {deploy:,} | {inc_deploy} |"
    )

header = "| Case | cost_per_state_byte | Cost of a new account | Cost increase | Cost of a new slot | Cost increase | 24kB contract deployment | Cost increase |"
sep = "|:---:|:---:|---|---|---|---|---|---|"
print(header)
print(sep)
for r in rows:
    print(r)

| Case | cost_per_state_byte | Cost of a new account | Cost increase | Cost of a new slot | Cost increase | 24kB contract deployment | Cost increase |
|:---:|:---:|---|---|---|---|---|---|
| Current cost | NA | 25,000 | NA | 20,000 | NA | 4,947,200 | NA |
| Proposed cost at 60M block limit | 662 | 74,144 | 3 | 24,084 | 1 | 16,357,064 | 3 |
| Proposed cost at 100M block limit | 1,174 | 131,488 | 5 | 40,468 | 2 | 28,997,320 | 6 |
| Proposed cost at 200M block limit | 2,198 | 246,176 | 10 | 73,236 | 4 | 54,277,832 | 11 |
| Proposed cost at 300M block limit | 3,222 | 360,864 | 14 | 106,004 | 5 | 79,558,344 | 16 |
